The idea is to get Trump's and Hillary's top 100 followers and find out which are their the top 100 accounts they follow. The goal is to find some similarities between them (or not, and to check the "social media bubble" theory).

# Script

In [1]:
import twitter as t
from twitter import TweepError

import redis
r = redis.StrictRedis(db=0)

import storage
s = storage.RedisStorage(r)

from datetime import datetime

select_user_data = lambda u: dict([(k, getattr(u, k)) for k in \
    ['screen_name', 'name', 'description', 'friends_count', 'followers_count']])


def load_user_data(user_id=None, screen_name=None):
    """
    Retrieve and set user's data.
    Or get it from the store if already there.
    """
    assert bool(user_id) != bool(screen_name)
    if user_id:
    user_data = s.get_user_data(user_id)
    if user_data:
            return user_id, user_data
    user = t.get_user(user_id=user_id)
    else: # screen_name
    user = t.get_user(screen_name=screen_name)
    user_id = user.id
    user_data = select_user_data(user)
    s.set_user_data(user_id, user_data)
    return user_id, user_data

def load_followers(user_id):
    """
    Retrieve and set user's followers.
    """
    followers = sorted(list(t.followers_ids(user_id)))
    s.set_followers(user_id, followers)
    # followers = s.get_followers(user_id)
    return followers

def load_friends(user_id):
    """
    Retrieve and set user's friends.
    """
    if s.is_protected(user_id) or \
       s.has_friends(user_id): # loaded before
        return
    try:
        friends = sorted(list(t.friends_ids(user_id)))
        s.set_friends(user_id, friends)
    except TweepError, e:
        if 'Not authorized' in str(e):
            s.mark_protected(user_id)

def aggregate_friends():
    "Aggregate friends into top most followed."
    s.set_most_followed()

    
#Not necessary but still...
def top_most_followed(n):
    """
    Display top n most followed.
    """
    i = 1
    top = s.get_most_followed(n) # withscores
    format = "%d | %d | %s | %s | %s ([@%s](https://twitter.com/%s))"
    print "Rank | Popularity | Followers | Friends | Name (@twitter)"
    print "--- | --- | --- | --- | ---"
    for user_id, score in top:
        user_id, user_data = load_user_data(user_id=user_id)
        print format % (i, score,
            user_data['followers_count'], user_data['friends_count'],
            user_data['name'],
            user_data['screen_name'], user_data['screen_name'])
        i += 1

IndentationError: expected an indented block (<ipython-input-1-b8e8ab5d9536>, line 22)

In [ ]:
####### FOR TRUMP
# Step 1: Identify the source, here we'll have 2 sources, Trump and Hillary
source_name = 'realDonaldTrump' # target group source Trump
source_id, source_data = load_user_data(screen_name=source_name)

# Step 2: Load target group members IDs
# Here we have to think about the time it will take
followers_ids = load_followers_ids(source_id) # target group

# Step 3: Load friends of target group members
for followers in followers_ids:
    load_friends(user_id=followers)

# Step 4: Aggregate friends into top most followed
aggregate_friends() # count friend occurrences
top_most_followed(100) # display results

In [ ]:
####### FOR TRUMP
# Step 1: Identify the source, here we'll have 2 sources, Trump and Hillary
source_name = 'HillaryClinton' # target group source Clinton
source_id, source_data = load_user_data(screen_name=source_name)

# Step 2: Load target group members IDs
followers_ids = load_followers_ids(source_id) # target group

# Step 3: Load friends of target group members
for followers in followers_ids:
    load_friends(user_id=followers)

# Step 4: Aggregate friends into top most followed
aggregate_friends() # count friend occurrences
top_most_followed(100) # display results